## Librerias


In [201]:
#Manejo de datos
import pandas as pd
import numpy as np


#Fuzz
!pip install thefuzz
from thefuzz import fuzz

#Uso de strings
import re
!pip install Unidecode
import unidecode  # Para remover tildes

#Google colab
from google.colab import auth
from google.cloud import bigquery

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Base

In [247]:
df1 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')
cities_df = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/cities.csv', usecols= ['city']) #Ciudades

### **Df 1**

- campaign: Nombre del evento enviada por web service
- customer: Nombre del comercio (eticket/tuboleta)
- creation_date: timestamp de la primera Alcancía creada

### **Df 2**
- campaign: Nombre del evento creada por Armatuvaca.
- campaign_slug: 	slug de las campañas.
- customer: Nombre del comercio (eticket/tuboleta)
- creation_date: timestamp de la fecha de creación del evento.


_________________________________________________________________________________________________________________________________________________________________


# **Analisis inicial**

Hasta el momento se han creado 348 eventos que aún no están diferenciados por campaña.

In [248]:
len(df1)

348

Exiten 136 campañas de tuboleta y 212 campañas de eticket

In [249]:
df1.groupby(['customer']).count()

,campaign,year_campaign
customer,,
Tuboleta,136,136
eticket,212,212


Hay 262 eventos dummy creados internamente, son relativamente menos a la info. enviada por las tiqueteras pues en ocasiones los eventos en eticket/tuboleta que tienen varios días se crean como eventos independientes, ejemplo: Festival Estereo Picnic sábado, Festival Estereo Picnic domingo; internamente se crean como un solo evento (Festival Estereo Picnic)

Además, en ocasiones internamente no se crea el dummy al evento por razones como:



*   Evento con muy pocas ventas
*   Evento muy próximo a realizarse



In [250]:
len(df2)

262

La manera de identificar que un evento es eticket o tuboleta es a través de 'dummy_redirect' sin embargo 4 eventos no tienen un link que redirige al check out **problema**

In [251]:
df2.groupby(['customer'])['slug'].count().reset_index()

,customer,slug
0,Armatuvaca,4
1,Tuboleta,115
2,eticket,143


In [252]:
df2.groupby(['customer', 'campaign'])['slug'].count().reset_index()

,customer,campaign,slug
0,Armatuvaca,CORE,1
1,Armatuvaca,Corasound Fest,1
2,Armatuvaca,Festival Estéreo Picnic 2023,1
3,Armatuvaca,La Solar,1
4,Tuboleta,&ME from KEINEMUSIK,1
...,...,...,...
218,eticket,WOS,1
219,eticket,Wild Brunch Cartagena,1
220,eticket,X BY ADRIATIQUE,1
221,eticket,YSY A,1


Teneindo en cuenta lo anterior, no es una relación uno a uno sino varios registros web service pueden tener un solo registro en 'piggys_piggycampaign'

# **Casos más comunes**



## 1.   **Muchas campañas un solo evento**
* BAUM FESTIVAL 2024  25 DE MAYO  SÁBADO INDIVIDUAL
* BAUM FESTIVAL 2024  25 DE MAYO SÁBADO INDIVIDUAL PROMOTORES
* BAUM FESTIVAL 2024  PROMOTORES
* BAUM FESTIVAL 2024 COMBO 2 DIAS
* BAUM FESTIVAL 2024 COMBO 2 DÍAS  PROMOTORES

Todos los eventos pertenecen a: **'Baum Festival'**, 'baum-festival-2024'
## 2.   **Similitud alta entre varios eventos siendo campañas independientes**

* CAMILO NUESTRO LUGAR FELIZ TOUR **-** Camilo **-** camilo-bogota-2024
* CAMILO NUESTRO LUGAR FELIZ TOUR  2DA FECHA **-** Camilo**-**camilo-bogota-2024-2

## 3. **Campañas que no fueron creadas como dummys**

JHONNY RIVERA  20 AÑOS

## 4. **Campañas para diferentes ciudades sin la etiqueta de la ciudad (Ej: Bogotá)**
* VIVE LA SALSA   PALCOS & GRADERIA - vive-la-salsa-bogota-2024 - Vive La Salsa
* VIVE LA SALSA  (COMBO 2 DÍAS) PALCOS & GRADERIA - vive-la-salsa-bogota-2024 - Vive La Salsa
* VIVE LA SALSA  CALI - vive-la-salsa-cali-2024 - Vive La Salsa
* VIVE LA SALSA  MEDELLÍN - vive-la-salsa-medellin-2024 - Vive La Salsa

## 5. **Campañas sin ningún tipo de coincidencia o bien muy poca**

* TOMORROWLAND PRESENTS CORE  2025  SINGLE - CORE - core-medellin-2025

## 6. **Campañas dummy no clasificadas como eticket/tuboleta**

**Problema extracción de la info.(dummy_redirect)**

* CORE
* Corasound Fest
* Festival Estéreo Picnic 2023
* La Solar

## 7. **Uso de Stopwords**

- "Promotores"
- DTO (Descuento)








## **Distancias de Levenshtein**

In [253]:
print(fuzz.ratio('hola', 'hoLA,')) #calcula la distancia de edición basándose en el ORDEN de ambas cadenas de entrada.
print(fuzz.partial_ratio('hola', 'hoLA,'))  #calcula la similitud tomando la cadena más corta y luego la compara con las subcadenas de la misma longitud de la cadena más larga, que se almacena en nombre_completo.
print(fuzz.token_sort_ratio('hola', 'HOLA,')) # Tiene en cuenta las cadenas similares que no están en el orden expresado anteriormente.
print(fuzz.token_set_ratio('hola', 'HOLA,')) # elimina los tokens comunes antes de calcular lo similares que son las cadenas: esto es extremadamente útil cuando las cadenas tienen una longitud significativamente diferente.
print(fuzz.partial_token_sort_ratio('hola', 'HOLA,'))# I guess, toma la frase más corta y no tiene en cuenta el orden de las palabras

44
67
100
100
100


# Metodología

In [ ]:
#Distanciasmás utilizadas
##Ratio = fuzz.ratio(d1.lower(),d2.lower())
##Partial_Ratio = fuzz.partial_ratio(d1.lower(),d2.lower())
##Token_Sort_Ratio = fuzz.token_sort_ratio(d1.lower(),d2.lower())
##Token_Set_Ratio = fuzz.token_set_ratio(d1.lower(),d2.lower())

In [314]:
stopwords = {"promotores", "dto", "fan days"}


def remove_stopwords(text):
    """
    Solo remueve stopwords sin modificar mayúsculas/minúsculas en el texto final.
    """
    words = str(text).split()
    filtered_words = [word for word in words if word.lower() not in stopwords]  # Convertimos a minúsculas solo para la comparación
    return " ".join(filtered_words)


In [315]:

def normalize_text(text):
    """
    Normaliza el texto: convierte a minúsculas y remueve tildes
    """
    text = str(text).lower()
    text = unidecode.unidecode(text)
    return text


In [316]:
def load_data():
    """
    Carga los archivos de datos y prepara las ciudades
    """
    print("Leyendo archivos...")
    df1 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base1.csv')
    df2 = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/base2.csv')
    cities_df = pd.read_csv('/content/drive/MyDrive/DATA/2024/11NOVIEMBRE/entity maching/cities.csv')

    df1['campaign'] = df1['campaign'].apply(remove_stopwords)

    for text in df1['campaign'].head():
        print(f"Normalizado: {text}")

    # Normalizar ciudades (sin eliminar stopwords para mantener nombres de ciudades intactos)
    cities = cities_df['city'].apply(lambda x: str(x).lower()).apply(unidecode.unidecode).unique()
    cities = sorted(cities, key=len, reverse=True)

    # Crear diccionario de ciudades normalizadas a originales
    cities_dict = dict(zip(cities_df['city'].apply(lambda x: str(x).lower()).apply(unidecode.unidecode),
                          cities_df['city']))

    return df1, df2, cities, cities_dict

def find_city_in_campaign(campaign_text, cities):
    """
    Busca si alguna ciudad de la lista está en el nombre de la campaña
    """
    # Para ciudades, solo aplicamos lowercase y unidecode, sin eliminar stopwords
    campaign_text = str(campaign_text).lower()
    campaign_text = unidecode.unidecode(campaign_text)

    for city in cities:
        if city in campaign_text:
            return city
    return None


In [317]:
def calculate_match_score(row1, row2, cities, cities_dict):
    """
    Calcula el score de match considerando el texto y la ciudad si existe
    """
    # Buscar ciudad en campaign de base1
    city_in_campaign = find_city_in_campaign(row1['campaign'], cities)

    # Si hay ciudad en el campaign, verificar que coincida con event_city de base2
    if city_in_campaign is not None:
        if 'event_city' in row2:
            event_city_normalized = str(row2['event_city']).lower()
            event_city_normalized = unidecode.unidecode(event_city_normalized)
            if event_city_normalized != city_in_campaign:
                return 0

    # Normalizar textos incluyendo eliminación de stopwords para la comparación
    campaign1 = normalize_text(row1['campaign'])
    campaign2 = normalize_text(row2['campaign'])

    # Calcular scores
    token_sort = fuzz.token_sort_ratio(campaign1, campaign2)
    token_set = fuzz.token_set_ratio(campaign1, campaign2)
    partial_ratio = fuzz.partial_ratio(campaign1, campaign2)

    return (token_sort + token_set + partial_ratio) / 3

In [318]:
def find_best_matches(df1, df2, cities, cities_dict):
    """
    Encuentra las mejores coincidencias considerando las ciudades cuando existan
    """
    all_possible_matches = []
    total_customers = len(df1['customer'].unique())
    print(f"\nProcesando {total_customers} customers...")

    for i, customer in enumerate(df1['customer'].unique(), 1):
        print(f"Procesando customer {i}: {customer}")

        df1_customer = df1[df1['customer'] == customer]
        df2_customer = df2[df2['customer'] == customer]

        for idx1, row1 in df1_customer.iterrows():
            # Buscar ciudad en el nombre de la campaña
            city_in_campaign = find_city_in_campaign(row1['campaign'], cities)

            # Filtrar df2 por año
            df2_year = df2_customer[df2_customer['year_campaign'] == row1['year_campaign']]
            if len(df2_year) == 0:
                df2_year = df2_customer

            for idx2, row2 in df2_year.iterrows():
                # Si hay ciudad en campaign, verificar coincidencia con event_city
                if city_in_campaign is not None:
                    event_city_normalized = normalize_text(row2['event_city']) if 'event_city' in row2 else ''
                    if event_city_normalized != city_in_campaign:
                        continue  # Saltar si las ciudades no coinciden

                match_score = calculate_match_score(row1, row2, cities, cities_dict)

                if match_score > 0:  # Solo agregar si hay posibilidad de match
                    all_possible_matches.append({
                        'customer': customer,
                        'year_campaign': row1['year_campaign'],
                        'campaign_df1': row1['campaign'],
                        'campaign_base1_id': idx1,
                        'campaign_df2': row2['campaign'],
                        'campaign_base2_id': idx2,
                        'campaign_slug': row2['slug'],
                        'city_in_campaign': cities_dict.get(city_in_campaign, city_in_campaign) if city_in_campaign else None,
                        'event_city': row2['event_city'] if 'event_city' in row2 else None,
                        'match_score': match_score
                    })

    if not all_possible_matches:
        raise ValueError("No se encontraron matches posibles entre las bases")

    # Convertir a DataFrame y ordenar
    all_matches_df = pd.DataFrame(all_possible_matches)
    all_matches_df = all_matches_df.sort_values('match_score', ascending=False)

    # Seleccionar mejores matches únicos
    used_base1 = set()
    #used_base2 = set()
    best_unique_matches = []

    for _, match in all_matches_df.iterrows():
        base1_id = match['campaign_base1_id']
        #base2_id = match['campaign_base2_id']

        if base1_id not in used_base1: #and base2_id not in used_base2:
            best_unique_matches.append(match)
            used_base1.add(base1_id)
            #used_base2.add(base2_id)

    results_df = pd.DataFrame(best_unique_matches)
    results_df = results_df.drop(['campaign_base1_id', 'campaign_base2_id'], axis=1)
    results_df = results_df.sort_values(['customer', 'match_score'], ascending=[True, False])

    return results_df

In [319]:
# Cargar datos
df1, df2, cities, cities_dict = load_data()

# Ejecutar el matching
print("Iniciando proceso de matching...")
matches = find_best_matches(df1, df2, cities, cities_dict)

# Filtrar coincidencias
good_matches = matches[matches['match_score'] >= 70]
suspicious_matches = matches[(matches['match_score'] >= 50) & (matches['match_score'] < 70)]
bad_matches = matches[matches['match_score'] < 50]

# Guardar resultados
good_matches.to_csv('matches_buenos.csv', index=False)
suspicious_matches.to_csv('matches_sospechosos.csv', index=False)
bad_matches.to_csv('matches_malos.csv', index=False)

# Imprimir resumen
print("\nResumen de resultados:")
print(f"Total de matches únicos: {len(matches)}")
print(f"Matches buenos (score >= 70): {len(good_matches)}")
print(f"Matches sospechosos (50-70): {len(suspicious_matches)}")
print(f"Matches malos (< 50): {len(bad_matches)}")

Leyendo archivos...
Normalizado: DUPLAT MOSAICO PIRATA
Normalizado: ROJUU
Normalizado: PIPE BUENO 3ER ROUND
Normalizado: CAMILO NUESTRO LUGAR FELIZ TOUR 2DA FECHA
Normalizado: VIVE LA SALSA CALI
Iniciando proceso de matching...

Procesando 2 customers...
Procesando customer 1: eticket
Procesando customer 2: Tuboleta

Resumen de resultados:
Total de matches únicos: 346
Matches buenos (score >= 70): 274
Matches sospechosos (50-70): 30
Matches malos (< 50): 42


In [322]:
matches

,customer,year_campaign,campaign_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
21625,Tuboleta,2024,RESIDENTE,Residente,residente-bogota-2024,None,Bogotá,100.000000
22378,Tuboleta,2024,MATTEO BOCELLI,Matteo Bocelli,matteo-bocelli-bogota-2024,None,Bogotá,100.000000
20145,Tuboleta,2025,JULIANA,Juliana,juliana-bogota-2025,None,Bogotá,100.000000
23980,Tuboleta,2025,DUKI,Duki,duki-bogota-2025,None,Bogotá,100.000000
17952,Tuboleta,2024,ENTONO Sound system,Entono Sound System,entono-bogota-2024,None,Bogotá,100.000000
...,...,...,...,...,...,...,...,...
14047,eticket,2023,MAJESTUOSOS 2023 23 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
15768,eticket,2023,MAJESTUOSOS 2023 30 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
6637,eticket,2023,MAJESTUOSOS 2023 28 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16610,eticket,2023,MAJESTUOSOS 2023 06 DE ENERO,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,20.000000


### Clasificados

In [320]:
good_matches = matches[matches['match_score'] >= 70]

In [321]:
good_matches

,customer,year_campaign,campaign_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
21625,Tuboleta,2024,RESIDENTE,Residente,residente-bogota-2024,None,Bogotá,100.000000
22378,Tuboleta,2024,MATTEO BOCELLI,Matteo Bocelli,matteo-bocelli-bogota-2024,None,Bogotá,100.000000
20145,Tuboleta,2025,JULIANA,Juliana,juliana-bogota-2025,None,Bogotá,100.000000
23980,Tuboleta,2025,DUKI,Duki,duki-bogota-2025,None,Bogotá,100.000000
17952,Tuboleta,2024,ENTONO Sound system,Entono Sound System,entono-bogota-2024,None,Bogotá,100.000000
...,...,...,...,...,...,...,...,...
11770,eticket,2024,KNTXT CHARLOTTE DE EITTEpromotores,Charlotte de Witte KNTXT,kntxt-charlotte-de-witte-medellin-2024,None,Medellín,74.333333
6407,eticket,2024,INTERPOL CELEBRANDO LOS DISCOS: TURN ON THE BR...,Interpol,interpol-medellin-2024,None,Medellín,74.000000
2019,eticket,2024,FEID EN BOGOTA,Gonzalo Ávila en Bogotá,gonzalo-avila-bogota-2024,Bogotá,Bogotá,71.666667
2942,eticket,2024,FESTIVAL X BY ADRIATIQUE,Baum Festival,baum-festival-2024,None,Bogotá,70.333333


274 clasificados
- **Bacilos**
- **Andres Calamaro** no se realizo la clasifiación por ciudad
- **Los tigres del norte**
- THE JUANPIS LIVE SHOW: LOS NOTO ASUSTADOS TEMPORADA 3	- The Juanpis Live Show: Los Noto Asustados Temporada 2
- VIVE LA SALSA (COMBO 2 DÍAS) PALCOS & GRADERIA	Vive La Salsa	vive-la-salsa-medellin-2024
- Camilo segunda fecha
- FESTIVAL X BY ADRIATIQUE



### No clasificados

In [323]:
bad_matches = matches[matches['match_score'] < 70]
bad_matches

,customer,year_campaign,campaign_df1,campaign_df2,campaign_slug,city_in_campaign,event_city,match_score
21935,Tuboleta,2023,PRIMAVERA PASS VIP,Primavera Sound 2023,primavera-sound-bogota-2023,None,Bogotá,67.000000
20045,Tuboleta,2023,PRIMAVERA PASS GENERAL,The Cure Primavera,the-cure-primavera,None,Bogotá,63.000000
23823,Tuboleta,2024,FELIPE PELÁEZ UN SUEÑO LLAMADO SINFÓNICO BOGOTÁ,Miguel Mateos Sinfónico,miguel-mateos-bogota-2024,Bogotá,Bogotá,57.333333
21823,Tuboleta,2024,ANIME 2DA FECHA,Anime: Concierto Otaku,anime-concierto-otaku-bogota-2024,None,Bogotá,51.666667
22941,Tuboleta,2025,MAS CERCA DE TI TOUR 2DA FECHA,SÚPER CONCIERTO FERIA DE MANIZALES,super-concierto-feria-de-manizales-2025,None,Manizales,50.333333
...,...,...,...,...,...,...,...,...
14047,eticket,2023,MAJESTUOSOS 2023 23 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
15768,eticket,2023,MAJESTUOSOS 2023 30 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
6637,eticket,2023,MAJESTUOSOS 2023 28 DE DICIEMBRE,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,22.666667
16610,eticket,2023,MAJESTUOSOS 2023 06 DE ENERO,Wild Brunch Cartagena,wild-brunch-cartagena-2023,None,Cartagena,20.000000


### Dummy event

In [313]:
matches.loc[matches['match_score'] <= 70, ['match_score', 'campaign_df2', 'campaign_slug']] = None
matches= matches[['campaign_df1', 'campaign_df2', 'campaign_slug', 'match_score']]
matches

,campaign_df1,campaign_df2,campaign_slug,match_score
21625,RESIDENTE,Residente,residente-bogota-2024,100.0
22378,MATTEO BOCELLI,Matteo Bocelli,matteo-bocelli-bogota-2024,100.0
20145,JULIANA,Juliana,juliana-bogota-2025,100.0
23980,DUKI,Duki,duki-bogota-2025,100.0
17952,ENTONO Sound system,Entono Sound System,entono-bogota-2024,100.0
...,...,...,...,...
14047,MAJESTUOSOS 2023 23 DE DICIEMBRE,None,None,NaN
15768,MAJESTUOSOS 2023 30 DE DICIEMBRE,None,None,NaN
6637,MAJESTUOSOS 2023 28 DE DICIEMBRE,None,None,NaN
16610,MAJESTUOSOS 2023 06 DE ENERO,None,None,NaN
